# Loader机制
- 加载markdown
- 加载csv
- 加载文件目录
- 加载html
- 加载json
- 加载pdf

In [1]:
#coding=utf8

# 使用loader来加载markdown文本

from dotenv import load_dotenv
load_dotenv()

from langchain.document_loaders import TextLoader
loader = TextLoader("loader.md")
loader.load()

[Document(page_content='# 我是一个markdown加载示例\n- 第一项目\n- 第二个项目\n- 第三个项目\n\n## 第一个项目\nAI研习社最厉害专业的AI研究基地\n\n## 第二个项目\nAIGC打造未来AI应用天地\n\n## 第三个项目\nAI研习社是一个非常牛逼的AI媒体', metadata={'source': 'loader.md'})]

In [6]:
# load cvs
from langchain.document_loaders import CSVLoader
loader = CSVLoader("loader.csv")
data = loader.load()
print(data[0].page_content)

Project: AI GC培训
DES: 培训课程
Price: 500
People: 100
Location: 北京


In [8]:
# 某个目录下有 excel文件，我们需要把目录下的文件加载进来
from langchain.document_loaders import DirectoryLoader

#目录下的。html 和。rst文件不会被这种load加载
#loader = DirectoryLoader(directory_path="", glob="*.csv")
loader = DirectoryLoader(path="./example", glob="*.xlsx")
docs = loader.load()
len(docs)

1

In [18]:
import json
from pathlib import Path
from typing import Callable, Dict, List, Optional, Union

from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader
class JSONLoader(BaseLoader):
    def __init__(
        self,
        file_path: Union[str, Path],
        content_key: Optional[str] = None,
        metadata_func: Optional[Callable[[dict, dict], dict]] = None,
    ):
        """
        Initializes the JSONLoader with a file path, an optional content key to extract specific content,
        and an optional metadata function to extract metadata from each record.
        """
        self.file_path = Path(file_path).resolve()
        self.content_key = content_key 
        self.metadata_func = metadata_func

    def create_documents(self, processed_data):
        """
        Creates Document objects from processed data.
        """
        documents = []
        for item in processed_data:
            content = item.get('content', '')  
            metadata = item.get('metadata', {})
            document = Document(page_content=content, metadata=metadata)
            documents.append(document)
        return documents

    def process_json(self, data):
        """
        Processes JSON data to prepare for document creation, extracting content based on the content_key
        and applying the metadata function if provided.
        """
        processed_data = []
        if isinstance(data, list):
            for item in data:
                content = item.get(self.content_key, '') if self.content_key else ''
                metadata = {}
                if self.metadata_func and isinstance(item, dict):
                    metadata = self.metadata_func(item, {})
                processed_data.append({'content': content, 'metadata': metadata})
        return processed_data

    def load(self) -> List[Document]:
        """
        Load and return documents from the JSON file.
        """
        docs = []
        with open(self.file_path, mode="r", encoding="utf-8") as json_file:
            try:
                data = json.load(json_file)
                processed_json = self.process_json(data)
                docs = self.create_documents(processed_json)
            except json.JSONDecodeError:
                print("Error: Invalid JSON format in the file.")
        return docs

In [12]:
# 使用loader来加载html文件
from langchain.document_loaders import UnstructuredHTMLLoader
# loader = UnstructuredHTMLLoader("loader.html")
from langchain.document_loaders import BSHTMLLoader
loader = BSHTMLLoader("loader.html")
data = loader.load()
data

[Document(page_content='RAG:将检索与生成方式相结合来做生成任务 - 知乎首发于自然语言处理算法与实践切换模式写文章登录/注册RAG:将检索与生成方式相结合来做生成任务烛之文\u200ba worker in NLP1、前言在上一篇<kNN-NER：利用knn近邻算法来做命名实体识别>提及到文中提出kNN-NER框架是一种检索式增强的方法（retrieval augmented methods），就去查看有关retrieval augmented的paper，了解其核心思想，觉得检索式增强的方法很适合许多业务场景使用，因其以一种简捷的方式将外部知识融于模型中去。今天就分享一篇来自Facebook AI Research的paper<Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks>，论文提出一种检索式增强生成方法，应用于知识密集型的NLP任务（如问答生成），该篇论文被2020年NeurIPS 会议接收。文中说到，以BERT之类的大规模预训练模型将很多事实知识信息存入模型中，可以看着是pre-trained parametric类型，尽管以fine-tuned方式在下游任务取得显著的成效，但这类方法仍存在无法精准地获取和操作知识的缺陷。而在上述提及的问题上，传统知识检索的方法能很好的应对，这类方法可以看着是non-parametric memory类型。于是，论文提出检索式增强生成方法（retrieval-augmented generation，RAG），主要思想就是将pre-trained parametric与non-parametric memory结合起来做语言生成任务，将两类模型集成起来提高任务处理效果。2、RAG方法上图为论文提出RAG模型的整体示意图。主要包括两大模块：一个检索器（Retriever， p_\\eta(z|x) ） + 一个生成器（Generator， p_\\theta(y_i|x,z,y_{1:i-1}) ）。前者包括query encoder和document index，分别负责query的编码和文档的索引；后者是一个seq2seq的生成模型。在检索中，使用的是最大内积搜索的方法（MIPS）来检索top-K相关文档。最后，把

In [22]:
#使用loader来加载JSON  https://github.com/langchain-ai/langchain/issues/4396
# from langchain.document_loaders import JSONLoader

loader = JSONLoader("../Chapter3/simple_prompts.json")
data = loader.load()
print(data)

[]


In [27]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("loader.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='蒂法介绍蒂法·洛克哈特（⽇语：ティファ・ロックハート，Tifa Rokkuhāto，英语：Tifa Lockhart）为电⼦游戏《最终幻想VII》及《最终幻想VII补完计划》相关作品中的虚构⻆⾊，由野村哲也创作和设计，此后也在多个游戏中客串登场。2014年东京电玩展上，星名美津纪cosplay《最终幻想VII 降临之⼦》中的蒂法·洛克哈特蒂法是克劳德的⻘梅⽵⻢，两⼈同为尼布鲁海姆出身。在⽶德加经营作为反抗组织“雪崩”根据地的酒馆“第七天堂”，并且是⼩有名⽓的招牌店员。擅⻓格⽃，以拳套为武器。本传7年前克劳德离开故乡从军时，曾许下约定“如果有危机时⼀定会保护她”。与爱丽丝相识之后，两⼈成为好友。第⼀个察觉克劳德记忆混乱的⼈，后来协助精神崩溃的克劳德重新找回真正的⾃⼰。本传的⼤战结束后，依⼤家的期待在战后新⽣的⽶德加再次开设第七天堂（原第七天堂因第柒区圆盘崩塌遭压毁），同时也照顾⼀群受到星痕症候群折磨的孩⼦们。蒂法被《纽约时报》称为“⽹络⼀代”的海报⼥郎，与劳拉·克罗夫特相⽐，她是电⼦游戏中坚强、独⽴和有吸引⼒的⼥性⻆⾊的典型代表。媒体普遍称赞其实⼒和外表，并称她为游戏世界中最好的⼥性⻆⾊之⼀。在《最终幻想VII》本传中，蒂法年龄20岁、身⾼167cm、⽣⽇5⽉3⽇、⾎型B型、出⽣地尼布尔海姆。登场《最终幻想VII》蒂法在《最终幻想VII》原版中⾸次亮相，是克劳德的⻘梅⽵⻢、第七天堂酒吧的看板娘、极端环境组织“雪崩”成员，该组织反抗巨型企业“神罗”因其⼤量抽取魔晄⽤作动⼒能源。在注意到克劳德的性格改变后，她说服克劳德加⼊雪崩，以密切关注他，并且跟随他追寻游戏中的对⼿萨菲罗斯。她⽆法阻⽌克劳德被萨菲罗斯操纵，在他的精神崩溃后，她帮助克劳德康复，并且两⼈意识到彼此间的相互感觉，最后与伙伴们⼀同击败了萨菲罗斯。[2]在闪回中可知，⼉时的蒂法⼀直是村中⼩孩的⼈⽓王。在⺟亲过世后，思念⺟亲的蒂法决定沿着⼩路⾛到他们故乡尼布尔海姆附近的⼀座⼭上，认为这样就能⻅到过世的⺟亲，原本跟着蒂法的其他⼩孩都在半路上因害怕⽽放弃，唯独克劳德仍坚定的在后⾯跟随，希望能在危机时保护蒂法。然⽽，他们俩都从⼭上跌落受伤，蒂法昏迷了⼀个星期，她的⽗亲认为克劳德对此负有责任[3]，甚为严令禁⽌克劳德再接近蒂法，但蒂法反⽽从此更在意克劳德，两⼈成为要好的玩伴。